In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Dataset_Skripsi_Processed.csv'  # Path absolut
df = pd.read_csv(file_path)

In [3]:
# Menampilkan data pertama untuk memastikan dataset terbaca dengan benar
pd.set_option('display.max_rows', None)
print(df)

    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
0             P01         G01  0.0  0.0        0.0       0.0
1             P01         G02  0.6  1.0        0.8       0.8
2             P01         G03  0.0  0.0        0.0       0.0
3             P01         G04  0.0  0.0        0.0       0.0
4             P01         G05  0.0  0.0        0.0       0.0
5             P01         G06  0.0  0.0        0.0       0.0
6             P01         G07  0.0  0.0        0.0       0.0
7             P01         G08  0.0  0.0        0.0       0.0
8             P01         G09  1.0  0.8        0.9       1.0
9             P01         G10  0.8  1.0        0.9       1.0
10            P01         G11  0.6  1.0        0.8       0.8
11            P01         G12  0.6  1.0        0.8       0.8
12            P01         G13  0.0  0.0        0.0       0.0
13            P01         G14  0.0  0.0        0.0       0.0
14            P01         G15  0.0  0.0        0.0       0.0
15            P01       

In [4]:
# Filter data untuk P09 saja
df_P09 = df[df['kode_penyakit'] == 'P09']  # Ambil data hanya untuk P09

In [5]:
# Menampilkan data yang diambil untuk P09
print("Data untuk P09:")
print(df_P09)

Data untuk P09:
    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
248           P09         G01  0.8  1.0        0.9       1.0
249           P09         G02  0.0  0.0        0.0       0.0
250           P09         G03  0.0  0.0        0.0       0.0
251           P09         G04  0.8  1.0        0.9       1.0
252           P09         G05  0.6  1.0        0.8       0.8
253           P09         G06  0.0  0.0        0.0       0.0
254           P09         G07  0.0  0.0        0.0       0.0
255           P09         G08  0.0  0.0        0.0       0.0
256           P09         G09  0.0  0.0        0.0       0.0
257           P09         G10  0.0  0.0        0.0       0.0
258           P09         G11  0.0  0.0        0.0       0.0
259           P09         G12  0.0  0.0        0.0       0.0
260           P09         G13  0.0  0.0        0.0       0.0
261           P09         G14  0.0  0.0        0.0       0.0
262           P09         G15  0.0  0.0        0.0       0.0
263     

In [6]:
# Set seed for reproducibility
np.random.seed(42)

## Inisialisasi Parameter

In [7]:
# Parameter PSO tetap
c1 = 2.0  # Koefisien akselerasi 1
c2 = 2.0  # Koefisien akselerasi 2
n_particles = 10  # Jumlah partikel
n_iterations = 10  # Jumlah iterasi

## Fungsi Fitness

In [8]:
# Fungsi fitness untuk mengukur sejauh mana posisi partikel mendekati cf_pakar
def fitness(position, cf_pakar):
    return np.mean((cf_pakar - position) ** 2, axis=1)  # MSE 

## Implementasi

In [9]:
# Fungsi untuk menjalankan PSO dengan parameter bobot inersia yang berbeda untuk setiap penyakit
def run_pso(cf_pakar, w_max, w_min):
    fitness_best_results = []
    
    for trial in range(1, 11):  # 10 uji coba
        # Inisialisasi kecepatan dan posisi partikel secara acak
        position = np.random.uniform(0.0, 1.0, (n_particles, len(cf_pakar)))  # Posisi partikel
        velocity = np.random.uniform(-0.1, 0.1, (n_particles, len(cf_pakar)))  # Kecepatan partikel

        # Inisialisasi pbest dan gbest
        pbest = position.copy()  # Set pbest awal sama dengan posisi partikel
        pbest_score = np.array([np.mean(np.abs(cf_pakar - p)) for p in position])  # Nilai fitness awal untuk pbest
        gbest = pbest[np.argmin(pbest_score)]  # gbest adalah posisi dengan nilai fitness terbaik
        gbest_score = np.min(pbest_score)  # Nilai fitness terbaik untuk gbest

        # Iterasi PSO
        for iteration in range(n_iterations):
            w = w_max - (w_max - w_min) * (iteration / n_iterations)  # Interpolasi w antara w_max dan w_min

            # Update posisi dan kecepatan setiap partikel
            for i in range(n_particles):
                r1 = np.random.rand(len(cf_pakar))  # Random number untuk C1
                r2 = np.random.rand(len(cf_pakar))  # Random number untuk C2

                # Update kecepatan
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - position[i]) + c2 * r2 * (gbest - position[i])

                # Update posisi
                position[i] += velocity[i]

                # Pastikan posisi berada dalam rentang 0 dan 1
                position[i] = np.clip(position[i], 0.0, 1.0)

            # Evaluasi fitness setiap partikel
            score = fitness(position, cf_pakar)

            # Update pbest dan gbest
            for i in range(n_particles):
                if score[i] < pbest_score[i]:
                    pbest[i] = position[i]  # Update pbest dengan posisi partikel yang lebih baik
                    pbest_score[i] = score[i]

            # Update gbest jika ditemukan pbest yang lebih baik
            gbest_score_new = np.min(pbest_score)
            if gbest_score_new < gbest_score:
                gbest = pbest[np.argmin(pbest_score)]  # Update gbest dengan posisi pbest terbaik
                gbest_score = gbest_score_new

        # Simpan hasil fitness terbaik untuk setiap uji coba
        fitness_best_results.append(gbest_score)

    # Mengembalikan hasil fitness terbaik untuk 10 uji coba
    return fitness_best_results

In [10]:
# Daftar penyakit yang ingin diuji (hanya P09 untuk sekarang)
penyakit_codes = ['P09']  # Fokus pada P09 saja

In [11]:
# Menyimpan hasil uji coba per penyakit
results = []

In [12]:
# Kombinasi bobot inersia yang diuji
w_max_values = [0.9, 0.8, 0.7]
w_min_values = [0.2, 0.3, 0.4]

In [13]:
# Fokus pada P09
for penyakit in penyakit_codes:
    cf_pakar = df[df['kode_penyakit'] == penyakit]['cf_pakar'].values
    for w_max in w_max_values:
        for w_min in w_min_values:
            # Jalankan PSO dan simpan hasil per percobaan
            fitness_results = run_pso(cf_pakar, w_max, w_min)
            
            # Simpan hasil per percobaan (trial) untuk w_max dan w_min yang diuji
            for trial_num, fitness_score in enumerate(fitness_results, 1):
                results.append({
                    'Penyakit': penyakit,
                    'ωmax': w_max,
                    'ωmin': w_min,
                    'Percobaan': trial_num,
                    'Fitness Terbaik': fitness_score
                })

In [14]:
# Membuat DataFrame untuk menampilkan hasil
df_results = pd.DataFrame(results)

pd.set_option('display.max_rows', 1000)

In [15]:
# Menampilkan hasil tabel per percobaan
print(df_results)

   Penyakit  ωmax  ωmin  Percobaan  Fitness Terbaik
0       P09   0.9   0.2          1         0.112433
1       P09   0.9   0.2          2         0.084154
2       P09   0.9   0.2          3         0.102405
3       P09   0.9   0.2          4         0.137301
4       P09   0.9   0.2          5         0.106267
5       P09   0.9   0.2          6         0.058939
6       P09   0.9   0.2          7         0.098398
7       P09   0.9   0.2          8         0.115922
8       P09   0.9   0.2          9         0.043579
9       P09   0.9   0.2         10         0.099794
10      P09   0.9   0.3          1         0.102517
11      P09   0.9   0.3          2         0.068926
12      P09   0.9   0.3          3         0.076762
13      P09   0.9   0.3          4         0.072717
14      P09   0.9   0.3          5         0.099584
15      P09   0.9   0.3          6         0.139671
16      P09   0.9   0.3          7         0.095190
17      P09   0.9   0.3          8         0.035336
18      P09 

In [16]:
# Menghitung rata-rata fitness terbaik untuk setiap kombinasi ωmax dan ωmin
avg_fitness_results = df_results.groupby(['Penyakit', 'ωmax', 'ωmin'])['Fitness Terbaik'].mean().reset_index()

In [17]:
# Menampilkan kombinasi terbaik per penyakit berdasarkan rata-rata fitness
print("\nKombinasi ωmax dan ωmin terbaik per penyakit:")
print(avg_fitness_results)


Kombinasi ωmax dan ωmin terbaik per penyakit:
  Penyakit  ωmax  ωmin  Fitness Terbaik
0      P09   0.7   0.2         0.089933
1      P09   0.7   0.3         0.097134
2      P09   0.7   0.4         0.098996
3      P09   0.8   0.2         0.107333
4      P09   0.8   0.3         0.090496
5      P09   0.8   0.4         0.093304
6      P09   0.9   0.2         0.095919
7      P09   0.9   0.3         0.092006
8      P09   0.9   0.4         0.087954


In [18]:
# Jika ingin menambahkan header dan data ke file yang sudah ada (append):
df_results.to_csv('pengujian_bobot_inersia_P09.csv', mode='a', header=False, index=False)
